In [2]:
import os
import zipfile
import random
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt

In [3]:
source_path = './PetImages'

source_path_dogs = os.path.join(source_path, 'Dog')
source_path_cats = os.path.join(source_path, 'Cat')

print(f"There are {len(os.listdir(source_path_dogs))} images of dogs.")
print(f"There are {len(os.listdir(source_path_cats))} images of cats.")

There are 12470 images of dogs.
There are 12491 images of cats.


In [4]:
# Define root directory
root_dir = 'cats-v-dogs'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

def create_train_val_dirs(root_path):
  
  list_dir = ['training', 'validation']
  sublist = ['cats', 'dogs']
  for i in list_dir:
    os.makedirs(os.path.join(root_path, i))
    for j in sublist:
      os.makedirs(os.path.join(root_path, i, j))
  
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [5]:
# Test create_train_val_dirs function

for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

cats-v-dogs/validation
cats-v-dogs/training
cats-v-dogs/validation/dogs
cats-v-dogs/validation/cats
cats-v-dogs/training/dogs
cats-v-dogs/training/cats


In [6]:
src_dir = os.listdir('PetImages/Cat')
train_data = round(len(src_dir)* .90)
validation_data = round(len(src_dir)* (1-0.9))
# train_data
validation_data

1249

In [8]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets
  
  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training
    
  Returns:
    None
  """

  src_dir = os.listdir(SOURCE_DIR)

  for fn in src_dir:
    if os.path.getsize(os.path.join(SOURCE_DIR, fn)) <= 0:
      print(fn + " is zero length so ignoring.")
      src_dir.remove(fn)
  
  train_data = round(len(src_dir)* SPLIT_SIZE)
  validation_data = round(len(src_dir)* (1-SPLIT_SIZE))

  train_sample = random.sample(src_dir[:train_data], len(src_dir[:train_data]))
  validation_sample = random.sample(src_dir[train_data:], len(src_dir[:validation_data]))

  [copyfile(os.path.join(SOURCE_DIR, fn),os.path.join(TRAINING_DIR,fn)) for fn in train_sample]

  [copyfile(os.path.join(SOURCE_DIR, fn),os.path.join(VALIDATION_DIR,fn)) for fn in validation_sample]

In [26]:
# Define paths
CAT_SOURCE_DIR = "PetImages/Cat/"
DOG_SOURCE_DIR = "PetImages/Dog/"

TRAINING_DIR = "cats-v-dogs/training/"
VALIDATION_DIR = "cats-v-dogs/validation/"

TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")

TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_CATS_DIR)) > 0:
  for file in os.scandir(TRAINING_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(TRAINING_DOGS_DIR)) > 0:
  for file in os.scandir(TRAINING_DOGS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_CATS_DIR)) > 0:
  for file in os.scandir(VALIDATION_CATS_DIR):
    os.remove(file.path)
if len(os.listdir(VALIDATION_DOGS_DIR)) > 0:
  for file in os.scandir(VALIDATION_DOGS_DIR):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .6


split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

# Check that the number of images matches the expected output

print(f"\n\nOriginal cat's directory has {len(os.listdir(CAT_SOURCE_DIR))} images")
print(f"Original dog's directory has {len(os.listdir(DOG_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_CATS_DIR))} images of cats for training")
print(f"There are {len(os.listdir(TRAINING_DOGS_DIR))} images of dogs for training")
print(f"There are {len(os.listdir(VALIDATION_CATS_DIR))} images of cats for validation")
print(f"There are {len(os.listdir(VALIDATION_DOGS_DIR))} images of dogs for validation")



Original cat's directory has 12491 images
Original dog's directory has 12470 images

There are 7495 images of cats for training
There are 7482 images of dogs for training
There are 4996 images of cats for validation
There are 4988 images of dogs for validation


In [27]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.utils.data import Dataset

In [28]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available; using CPU")

GPU is available


In [29]:
# Define data transformations (e.g., resizing and normalization)

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop((224,224)),  # Randomly crop and resize
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.RandomRotation(degrees=15),  # Random rotation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224,224)),  # Resize without augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
path_dataset = './petImages/'
train_dataset = ImageFolder(root=TRAINING_DIR, transform=train_transforms)
test_dataset = ImageFolder(root=VALIDATION_DIR, transform=test_transforms)

# Create data loaders for training and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)


In [30]:
import torch.nn as nn

class CatDogClassifier(nn.Module):
    def __init__(self):
        super(CatDogClassifier, self).__init__()
        # Define the layers for your CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(in_features=32*56*56, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=2)  # 2 classes (cat and dog)

    def forward(self, x):
        # Define the forward pass
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the CatDogClassifier
model = CatDogClassifier().to(device)


In [31]:
import torch.optim as optim

# Define the loss function
criterion = nn.CrossEntropyLoss().to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [32]:
from tqdm import tqdm

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # Use tqdm to create a progress bar
    with tqdm(train_loader, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            t.set_postfix(loss=running_loss / len(train_loader))
    
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

100%|██████████| 234/234 [02:09<00:00,  1.80batch/s, loss=0.736]


Epoch 1, Loss: 0.735877191918528


100%|██████████| 234/234 [02:22<00:00,  1.65batch/s, loss=0.649]


Epoch 2, Loss: 0.6490423895864406


100%|██████████| 234/234 [02:36<00:00,  1.50batch/s, loss=0.639]


...

Epoch 8, Loss: 0.5896404548587962


100%|██████████| 234/234 [02:11<00:00,  1.78batch/s, loss=0.579]


Epoch 9, Loss: 0.5786339015278041


100%|██████████| 234/234 [02:14<00:00,  1.74batch/s, loss=0.572]

Epoch 10, Loss: 0.5720885211331213
Finished Training


In [33]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

Accuracy on the test dataset: 74.42%
